In [1]:
import os

# Set both for redundancy
os.environ["HF_HOME"] = "/gpfs3/well/papiez/users/hri611/.cache/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/gpfs3/well/papiez/users/hri611/.cache/huggingfacee"
os.environ["HF_DATASETS_CACHE"] = "/gpfs3/well/papiez/users/hri611/.cache/huggingface/datasets"
os.environ["HF_METRICS_CACHE"] = "/gpfs3/well/papiez/users/hri611/.cache/huggingface/metrics"


In [ ]:
import io

import requests
import torch
import transformers
import glob
from pathlib import Path
import pydicom
import numpy as np
import io
import requests
from PIL import Image
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig, AutoTokenizer
import pandas as pd


/gpfs3/well/papiez/users/hri611/python/foundation-models-radiology/chexagent_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/gpfs3/well/papiez/users/hri611/python/foundation-models-radiology/chexagent_env/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-04-18 18:41:27.731897: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-18 18:41:30.201090: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register c

In [ ]:
# step 1: Setup constant
device = "cuda"
dtype = torch.float16

# step 2: Load Processor and Model
processor = AutoProcessor.from_pretrained("StanfordAIMI/CheXagent-8b", trust_remote_code=True)
generation_config = GenerationConfig.from_pretrained("StanfordAIMI/CheXagent-8b")
model = AutoModelForCausalLM.from_pretrained("StanfordAIMI/CheXagent-8b", torch_dtype=dtype, trust_remote_code=True)


Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:42<00:00, 14.60s/it]


In [ ]:
root_dir = Path('/well/papiez/users/hri611/python/foundation-models-radiology')
dicom_paths = glob.glob(str(root_dir / 'PTX Head to Head Study Data' / '**/*.dcm'), recursive=True)

def dicom_to_rgb_image(dicom_path):
    dicom_image = pydicom.dcmread(dicom_path)
    pixel_array = dicom_image.pixel_array

    # Normalize to 0-255
    pixel_array = (pixel_array - np.min(pixel_array)) / (np.max(pixel_array) - np.min(pixel_array) + 1e-5)
    pixel_array = (pixel_array * 255.0).astype(np.uint8)

    # Convert grayscale to RGB
    if pixel_array.ndim == 2:
        image = Image.fromarray(pixel_array).convert("RGB")
    elif pixel_array.ndim == 3 and pixel_array.shape[-1] == 3:
        image = Image.fromarray(pixel_array)
    else:
        raise ValueError(f"Unexpected DICOM shape: {pixel_array.shape}")
    
    # save as jpg
    jpg_path = dicom_path.replace('.dcm', '.jpg')
    image.save(jpg_path, "JPEG")

# images = [dicom_to_rgb_image(dicom_paths[0])]
for dicom in dicom_paths:
    dicom_to_rgb_image(dicom)


In [ ]:
# 8B Model
# # step 3: Fetch the images
# # image_path = 'Pleural_effusion-Metastatic_breast_carcinoma_Case_166_(5477628658).jpg'
# # images = [Image.open(image_path).convert("RGB")]

# # step 4: Generate the Findings section
# dtype = model.dtype
# model.to(device)

# prompt = f'Does this chest X-ray contain a pneumothorax?'
# inputs = processor(images=images, text=f" USER: <s>{prompt} ASSISTANT: <s>", return_tensors="pt").to(device=device, dtype=dtype)
# output = model.generate(**inputs, generation_config=generation_config)[0]
# response = processor.tokenizer.decode(output, skip_special_tokens=True)
# print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


No


In [ ]:
# step 1: Setup constant
model_name = "StanfordAIMI/CheXagent-2-3b"
dtype = torch.bfloat16
device = "cuda"

# step 2: Load Processor and Model
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)
model = model.to(device=device,dtype=dtype)
model.eval()

/gpfs3/well/papiez/users/hri611/python/foundation-models-radiology/chexagent_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/StanfordAIMI/CheXagent-2-3b:
- tokenization_chexagent.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
A new version of the following files was downloaded from https://huggingface.co/StanfordAIMI/CheXagent-2-3b:
- configuration_chexagent.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versi

CheXagentForCausalLM(
  (model): CheXagentModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layernorm): LayerN

In [ ]:
responses = []
for i in range(len(jpg_paths)):  
    query = tokenizer.from_list_format([{'image': jpg_paths[i]},{'text': 'Does this chest X-ray contain a pneumothorax?'}])
    conv = [{"from": "system", "value": "You are a helpful assistant."}, {"from": "human", "value": query}]
    input_ids = tokenizer.apply_chat_template(conv, add_generation_prompt=True, return_tensors="pt")
    output = model.generate(
        input_ids.to(device), do_sample=False, num_beams=1, temperature=1., top_p=1., use_cache=True,
        max_new_tokens=512
    )[0]
    response = tokenizer.decode(output[input_ids.size(1):-1])
    responses.append(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [ ]:
chexagent_df = pd.DataFrame({'image_path': jpg_paths, 'response': responses})

In [15]:
chexagent_df['binary_score'] = chexagent_df['response'].apply(lambda x: 1 if 'yes' in x.lower() else 0)
chexagent_df.drop(columns=['response'], inplace=True)
chexagent_df.to_csv('ptx_chexagent_scores.csv', index=False)


In [17]:
microsoft_df = pd.read_csv('ptx_biomedclip_scores.csv')
google_df = pd.read_csv('ptx_google_scores.csv')

merged_df = pd.merge(microsoft_df, google_df, on='image_paths', suffixes=('_microsoft', '_google'))

In [18]:
merged_df

,image_paths,score_microsoft,binary_score_microsoft,score_google,binary_score_google
0,c24f6df1417b47c32e8cd6a1602340c2/cdae392a3011b...,0.845350,1,0.464604,0
1,7ad7363df5636af3afe1aada18b3c5d3/2dbff6a398ac9...,0.445231,0,0.472215,0
2,cacfc2670b7a27f2e125494659bd12a0/cfe3de2280f48...,0.054741,0,0.451168,0
3,1425254d69091cb945ddbad9fd6af789/7388d9c06db3c...,0.821323,1,0.427989,0
4,5a2ec5990263c3dafa9812c4d6d33b2f/37c8395d71977...,0.570689,1,0.506624,1
...,...,...,...,...,...
408,3969fe0e0cc8b0956642b6a061a279bc/9c0c16260d8ce...,0.066489,0,0.521897,1
409,ffac9214400053aa4c78126255328235/113cd0ccb7393...,0.078097,0,0.493282,0
410,df25ba54718a67f9024f48bf03617eae/4d871a6f6c734...,0.165823,0,0.458779,0
411,ab73931f768d9132ad49af3442d364c1/9c24fd7c352e2...,0.016506,0,0.526047,1


In [19]:
merged_df['chexagent_scores'] = chexagent_df['binary_score']

In [20]:
merged_df

,image_paths,score_microsoft,binary_score_microsoft,score_google,binary_score_google,chexagent_scores
0,c24f6df1417b47c32e8cd6a1602340c2/cdae392a3011b...,0.845350,1,0.464604,0,1
1,7ad7363df5636af3afe1aada18b3c5d3/2dbff6a398ac9...,0.445231,0,0.472215,0,0
2,cacfc2670b7a27f2e125494659bd12a0/cfe3de2280f48...,0.054741,0,0.451168,0,0
3,1425254d69091cb945ddbad9fd6af789/7388d9c06db3c...,0.821323,1,0.427989,0,0
4,5a2ec5990263c3dafa9812c4d6d33b2f/37c8395d71977...,0.570689,1,0.506624,1,1
...,...,...,...,...,...,...
408,3969fe0e0cc8b0956642b6a061a279bc/9c0c16260d8ce...,0.066489,0,0.521897,1,1
409,ffac9214400053aa4c78126255328235/113cd0ccb7393...,0.078097,0,0.493282,0,1
410,df25ba54718a67f9024f48bf03617eae/4d871a6f6c734...,0.165823,0,0.458779,0,0
411,ab73931f768d9132ad49af3442d364c1/9c24fd7c352e2...,0.016506,0,0.526047,1,1


In [24]:
merged_df[['binary_score_google','binary_score_microsoft','chexagent_scores']].value_counts()

binary_score_google  binary_score_microsoft  chexagent_scores
0                    0                       0                   165
                                             1                    84
                     1                       0                    66
                                             1                    32
1                    0                       1                    28
                                             0                    19
                     1                       1                    17
                                             0                     2
Name: count, dtype: int64